# Tweets classification by its hashtags and no labeled tweets hashtag predication


**Using top n hashtags as label to build a supervised model for tweets classification and hashtag predication**

[1.1 load data](1.1)

## <a id='1.1'> load packages and modeling data </a>

In [1]:
#import tensorflow backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from keras.callbacks import EarlyStopping, ModelCheckpoint

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12881373835584278877
]


Using TensorFlow backend.


In [49]:
import keras
from keras.layers import *
from keras.models import Sequential, Model, load_model
from keras.optimizers import *
from keras.preprocessing import sequence

print(keras.__version__)
print(keras.backend.backend())
import keras.backend as K

2.1.2
tensorflow


In [3]:
import numpy as np
import bcolz
import pickle

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

def save_dict(fname, dictionary):
    with open(fname, 'wb') as f:
        pickle.dump(dictionary, f)

def load_dict(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

In [4]:
#load tweets data
import json
tweets_file = "temp/tweets4classification.json"
with open(tweets_file, "r", encoding="utf-8") as f:
    modeling_data = json.load(f)
modeling_data['data'][0]

{'hashtag_label': [1, 5],
 'hashtags': ['hpv', 'vaccin'],
 'id': '418263863772327936',
 'orignal_hashtags': ['#hpv', '#vaccine'],
 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…',
 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>',
 'words': ['rt',
  ':',
  'hpv',
  'vax',
  'coverage',
  'could',
  'be',
  '93',
  '%',
  'if',
  'doctors',
  'gave',
  'hpv',
  'vaccine',
  'each',
  'time',
  'a',
  'preteen',
  '/',
  'teen',
  'got',
  'any',
  'other',
  'vaccine',
  '>']}

## process word embeddings 

In [5]:
#load processed word enbeddings
path = 'wordsenbeddings/'
res_path = path + 'results/'

def load_vectors(name):
    loc = res_path + name
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [7]:
def get_glove(name, dim):
    with open(path+ 'glove.' + name + '.txt', 'r', encoding="utf-8") as f:
        vecs = []
        words = []
        
        for i, line in enumerate(f):
            d = line.split()
            word = d[0]
            vec = np.array(d[1:], dtype=np.float32)
            if (len(d) == dim): # this is space
                word = ' '
                vec = np.array(d, dtype=np.float32)
            
            words.append(word)            
            vecs.append(vec)

        wordidx = {o:i for i,o in enumerate(words)}
        save_array(res_path+name+'.dat', vecs)
        pickle.dump(words, open(res_path+name+'_words.pkl','wb'))
        pickle.dump(wordidx, open(res_path+name+'_idx.pkl','wb'))

In [7]:
get_glove('twitter.27B.200d', 200)
get_glove('twitter.27B.25d', 25)
get_glove('twitter.27B.50d', 50)
get_glove('twitter.27B.100d', 100)

## prepare train and test sample

In [6]:
data = np.asarray([each['words'] for each in modeling_data['data']])
label = np.asarray([each['hashtag_label'] for each in modeling_data['data']])

print(data[:2])
print(label[:2])
print(len(data))
print(len(label))

[ list(['rt', ':', 'hpv', 'vax', 'coverage', 'could', 'be', '93', '%', 'if', 'doctors', 'gave', 'hpv', 'vaccine', 'each', 'time', 'a', 'preteen', '/', 'teen', 'got', 'any', 'other', 'vaccine', '>'])
 list(['rt', ':', 'hpv', 'vax', 'coverage', 'could', 'be', '93', '%', 'if', 'doctors', 'gave', 'hpv', 'vaccine', 'each', 'time', 'a', 'preteen', '/', 'teen', 'got', 'any', 'other', 'vaccine', '...', '.'])]
[list([1, 5]) list([1, 5])]
81049
81049


In [7]:
def flat_labels(labels):
    flatted = []
    l = modeling_data['categorical_num']
    for label in labels:
        m = [0.] * l
        for each in label:
            m = list(map(lambda x: x[0] or x[1], zip(m, each)))
        flatted.append(m)
    return np.asarray(flatted)    

In [8]:
from keras.utils.np_utils import to_categorical
categorical_label = list(map(lambda x: to_categorical(x, num_classes=modeling_data['categorical_num']), label))

categorical_label_flatted = flat_labels(categorical_label)

print(len(categorical_label))
categorical_label_flatted[:20]

81049


array([[ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0

In [9]:
from sklearn.model_selection import train_test_split
X_train_0, X_test, y_train_0, y_test = train_test_split(data, categorical_label_flatted, test_size=0.2, random_state=42)

In [10]:
print(X_train_0[:2])
print(X_test[:2])
print(y_train_0[:10])
print(y_test[:10])

[ list(['rt', ':', 'new', 'infographic', 'on', 'how', 'most', 'cases', 'of', 'cervicalcancer', 'can', 'be', 'prevented', 'w', '/', 'tests', '&', 'hpv', 'vaccine', '.', 'vitalsigns'])
 list(['check', 'out', 'the', 'gci', 'team', "'s", 'newest', 'publication', 'on', 'hpv', 'vaccine', 'implementation', 'for', 'cancer', 'prevention', 'in', 'latinamerica', '!'])]
[ list(['two', 'uk', 'girls', 'left', 'paralyzed', 'after', 'hpv', 'jabs', '.', 'authorities', 'still', 'claim', 'it', "'s", 'coincidence', '.'])
 list(['cervicalcancer', 'deaths', 'have', 'decreased', 'dramatically', 'over', 'the', 'past', '40', 'years', ',', 'mostly', 'due', 'to', 'increased', 'screening', '.'])]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  1.  1.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0

In [11]:
#save 5% train samples to predication task
cut = int(len(X_train_0) * 0.95)
X_train = X_train_0[:cut]
y_train = y_train_0[:cut]
X_pred = X_train_0[cut:]
y_pred = y_train_0[cut:]

In [12]:
#!!!!!do not run this, run the third one below this to directly load the dictionary
#create words dictionary for the data
from functools import reduce

from collections import defaultdict
frequency = defaultdict(int)

all_tokens = list(reduce(lambda x, y: x + y, [l['words'] for l in modeling_data['data']]))
print(all_tokens[:5])

In [ ]:
print(len(all_tokens))

for token in all_tokens:
    frequency[token] += 1
    
dictionary = sorted(frequency.items(), key=lambda x:x[1], reverse=True)
dictionary = [k for k,v in dictionary[:vocab_size]]
print(dictionary[:10])
len(dictionary)

In [ ]:
save_dict('model/dict.dd', dictionary)

In [13]:
vocab_size = 0
dictionary = load_dict('model/dict.dd')
if (not vocab_size):
    vocab_size = len(dictionary)
dictionary[:20]

[':',
 'hpv',
 'rt',
 'vaccine',
 '.',
 'the',
 ',',
 'of',
 'to',
 'cancer',
 'in',
 'gardasil',
 'for',
 'cervicalcancer',
 'a',
 '&',
 'and',
 'is',
 '!',
 'cervical']

In [78]:
#word to index
X_train_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_train]
X_test_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_test]
X_pred_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_pred]

In [15]:
print(X_train_f[:5])
print(X_test_f[:5])
print(X_pred_f[:5])

[[2, 0, 46, 781, 22, 74, 100, 121, 7, 13, 35, 55, 352, 54, 40, 162, 15, 1, 3, 4, 1126], [364, 93, 5, 10430, 1019, 20, 1770, 3765, 22, 1, 3, 1637, 12, 9, 61, 10, 8726, 18], [1159, 49, 251, 22, 97, 4382, 12, 448, 3863, 8, 20, 1, 25, 493, 18, 400, 13], [2, 0, 122, 312, 90, 662, 57, 8, 369, 33, 13, 4, 529, 26, 967, 16, 1777, 831, 8, 39, 365, 12, 1], [2, 0, 27, 0, 11, 131, 19, 328, 10, 31, 244, 54, 40, 1, 51]]
[[210, 338, 41, 215, 165, 34, 1, 471, 4, 459, 182, 409, 29, 20, 411, 4], [13, 180, 57, 1813, 2238, 296, 5, 1477, 1003, 134, 6, 3875, 616, 8, 448, 67, 4], [2, 0, 27, 0, 14, 262, 7, 346, 362, 10, 14, 509, 110, 118, 250, 34, 168, 1, 3], [726, 12, 8803, 423, 23, 155, 15, 281, 223, 397, 8, 1, 0, 492, 287], [97, 1, 16, 9, 153, 2965, 3326, 3551, 22, 2585, 193, 18, 1077, 125, 8, 159, 0]]
[[3, 1413, 264, 41, 177, 1, 68, 32, 757, 37, 697], [2, 0, 4, 84, 981, 18, 11, 3379, 6, 226, 2928, 10352, 1760, 81, 10353, 6, 4817, 19993, 4], [2, 0, 30, 24, 478, 11, 11792, 21], [1, 25, 68, 569, 145, 917, 41,

## embedding words using GloVe

In [16]:
lens = np.array([len(doc) for doc in X_train_f])
(lens.max(), lens.min(), lens.mean())

(39, 2, 17.012127213987693)

In [114]:
seq_len = 39
embedding_dim = 200
vecs, words, wordidx = load_vectors('twitter.27B.%dd'%(embedding_dim))

In [115]:
X_train_u = sequence.pad_sequences(X_train_f, maxlen=seq_len)
X_test_u = sequence.pad_sequences(X_test_f, maxlen=seq_len)
X_pred_u = sequence.pad_sequences(X_pred_f, maxlen=seq_len)

In [116]:
print(X_train_u[:5])
print(X_test_u[:5])
print(X_pred_u[:5])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     2     0    46   781    22    74
    100   121     7    13    35    55   352    54    40   162    15     1
      3     4  1126]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   364    93     5
  10430  1019    20  1770  3765    22     1     3  1637    12     9    61
     10  8726    18]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0  1159    49
    251    22    97  4382    12   448  3863     8    20     1    25   493
     18   400    13]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     2     0   122   312    90   662    57     8
    369    33    13     4   529    26   967    16  1777   831     8    39
    365    12     1]
 [    0     0     0     0   

In [117]:
def create_embedding(dictionary):
    print(vecs.shape)
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i, word in enumerate(dictionary):
        #if word:# and re.match(r"^[a-zA-Z0-9\-]*$", word):
        src_idx = wordidx[word] if word in wordidx else 0
        
        if src_idx:
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = np.random.normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = np.random.normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [118]:
embedding = create_embedding(dictionary)

(1193517, 200)


## create NN mode

### multi-label model

In [142]:
#pure cnn model
dth = 0.5
filter_num = 5

model = Sequential()

#embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[embedding], trainable=True))
# model.add(Dropout(dth))

#hidden layer
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(dth))
# model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
# model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
# model.add(Dropout(dth))
# model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
# model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
# model.add(Dropout(dth))

#output layer
model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(dth))
model.add(Dense(modeling_data['categorical_num'], activation='softmax'))

#compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 39, 200)           4661400   
_________________________________________________________________
conv1d_54 (Conv1D)           (None, 39, 200)           200200    
_________________________________________________________________
conv1d_55 (Conv1D)           (None, 39, 200)           200200    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 200)           0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 20, 200)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 4000)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 11)                44011     
Total para

In [143]:
%%time
# model.optimizer.lr=1e-4
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# early_stopping = EarlyStopping(monitor='val_acc', patience=10)

sub = "2cnn+max+drop"
model_chk_path = 'model/cnn_hashtagcls_emb{}_weights{}.h5'.format(embedding_dim, sub)
mcp = ModelCheckpoint(model_chk_path, monitor="val_loss", verbose=1,
                      save_best_only=True, save_weights_only=False)
model.fit(X_train_u, y_train, validation_data=(X_test_u, y_test), epochs=500, batch_size=64, callbacks=[early_stopping, mcp])

Train on 61597 samples, validate on 16210 samples
Epoch 1/500
61597/61597 [==============================] - 356s 6ms/step - loss: 1.4944 - acc: 0.7398 - val_loss: 1.2594 - val_acc: 0.6869
Epoch 2/500
61597/61597 [==============================] - 344s 6ms/step - loss: 1.2197 - acc: 0.7627 - val_loss: 1.2147 - val_acc: 0.7394
Epoch 3/500
61597/61597 [==============================] - 343s 6ms/step - loss: 1.1533 - acc: 0.7659 - val_loss: 1.2184 - val_acc: 0.7535
Epoch 4/500
61597/61597 [==============================] - 345s 6ms/step - loss: 1.1184 - acc: 0.7690 - val_loss: 1.2150 - val_acc: 0.7804
Epoch 5/500
61597/61597 [==============================] - 343s 6ms/step - loss: 1.0952 - acc: 0.7695 - val_loss: 1.2142 - val_acc: 0.7131
Epoch 6/500
61597/61597 [==============================] - 344s 6ms/step - loss: 1.0812 - acc: 0.7680 - val_loss: 1.2081 - val_acc: 0.7163
Epoch 7/500
61597/61597 [==============================] - 343s 6ms/step - loss: 1.0701 - acc: 0.7689 - val_loss: 1.

In [122]:
#pure cnn model
dth = 0.5
filter_num = 5

model = Sequential()

#embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[embedding], trainable=True))
# model.add(Dropout(dth))

#hidden layer
model.add(Conv1D(embedding_dim, filter_num, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(Conv1D(embedding_dim, filter_num, padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv1D(embedding_dim, filter_num, padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv1D(embedding_dim, filter_num, padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv1D(embedding_dim, filter_num, padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))

#output layer
model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(dth))
model.add(Dense(modeling_data['categorical_num'], activation='softmax'))

#compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 39, 200)           4661400   
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 39, 200)           200200    
_________________________________________________________________
batch_normalization_18 (Batc (None, 39, 200)           800       
_________________________________________________________________
activation_18 (Activation)   (None, 39, 200)           0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 39, 200)           200200    
_________________________________________________________________
batch_normalization_19 (Batc (None, 39, 200)           800       
_________________________________________________________________
activation_19 (Activation)   (None, 39, 200)           0         
__________

In [123]:
%%time
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# early_stopping = EarlyStopping(monitor='val_acc', patience=10)

# sub = "used_for_prediction_bn"
sub = "2cnn_bn_ac"
model_chk_path = 'model/cnn_hashtagcls_emb{}_weights{}.h5'.format(embedding_dim, sub)
mcp = ModelCheckpoint(model_chk_path, monitor="val_loss", verbose=1,
                      save_best_only=True, save_weights_only=False)
model.fit(X_train_u, y_train, validation_data=(X_test_u, y_test), epochs=500, batch_size=64, callbacks=[early_stopping, mcp])

Train on 61597 samples, validate on 16210 samples
Epoch 1/500
61597/61597 [==============================] - 390s 6ms/step - loss: 1.5777 - acc: 0.7228 - val_loss: 1.3855 - val_acc: 0.7319
Epoch 2/500
61597/61597 [==============================] - 381s 6ms/step - loss: 1.2565 - acc: 0.7597 - val_loss: 1.2883 - val_acc: 0.6893
Epoch 3/500
61597/61597 [==============================] - 384s 6ms/step - loss: 1.1616 - acc: 0.7664 - val_loss: 1.2351 - val_acc: 0.7563
Epoch 4/500
61597/61597 [==============================] - 386s 6ms/step - loss: 1.1191 - acc: 0.7689 - val_loss: 1.2407 - val_acc: 0.7532
Epoch 5/500
61597/61597 [==============================] - 384s 6ms/step - loss: 1.0920 - acc: 0.7687 - val_loss: 1.2344 - val_acc: 0.7721
Epoch 6/500
61597/61597 [==============================] - 384s 6ms/step - loss: 1.0748 - acc: 0.7705 - val_loss: 1.2135 - val_acc: 0.7458
Epoch 7/500
61597/61597 [==============================] - 384s 6ms/step - loss: 1.0626 - acc: 0.7700 - val_loss: 1.

In [133]:
#cnn Bi-LSTM model
dth = 0.5
filter_num = 5
model = Sequential()

#embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[embedding], trainable=False))
# model.add(Dropout(dth))

#hidden layers
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Dropout(dth))
model.add(Bidirectional(LSTM(embedding_dim, dropout=0.5, recurrent_dropout=0.5)))
# model.add(Bidirectional(LSTM(embedding_dim,  batch_input_shape=(64, 39, 50), stateful=True)))
# model.add(Dropout(dth))

#output layer
model.add(Dense(modeling_data['categorical_num'], activation='softmax'))

#compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 39, 200)           4661400   
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 39, 200)           200200    
_________________________________________________________________
dropout_27 (Dropout)         (None, 39, 200)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 400)               641600    
_________________________________________________________________
dense_15 (Dense)             (None, 11)                4411      
Total params: 5,507,611
Trainable params: 846,211
Non-trainable params: 4,661,400
_________________________________________________________________


In [134]:
# model.optimizer.lr=1e-4
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# early_stopping = EarlyStopping(monitor='val_acc', patience=10)

sub = "cnn_biLSTM"
model_chk_path = 'model/cnn_hashtagcls_emb{}_weights{}.h5'.format(embedding_dim, sub)
mcp = ModelCheckpoint(model_chk_path, monitor="val_loss", verbose=1,
                      save_best_only=True, save_weights_only=False)
model.fit(X_train_u, y_train, validation_data=(X_test_u, y_test), epochs=500, batch_size=64, callbacks=[early_stopping, mcp])

Train on 61597 samples, validate on 16210 samples
Epoch 1/500
61597/61597 [==============================] - 1109s 18ms/step - loss: 1.6367 - acc: 0.7299 - val_loss: 1.3020 - val_acc: 0.8057
Epoch 2/500
61597/61597 [==============================] - 1091s 18ms/step - loss: 1.3162 - acc: 0.7597 - val_loss: 1.2374 - val_acc: 0.7546
Epoch 3/500
61597/61597 [==============================] - 1086s 18ms/step - loss: 1.2596 - acc: 0.7613 - val_loss: 1.2171 - val_acc: 0.8152
Epoch 4/500
61597/61597 [==============================] - 1081s 18ms/step - loss: 1.2328 - acc: 0.7620 - val_loss: 1.1955 - val_acc: 0.7759
Epoch 5/500
61597/61597 [==============================] - 1084s 18ms/step - loss: 1.2121 - acc: 0.7631 - val_loss: 1.1929 - val_acc: 0.6832
Epoch 6/500
61597/61597 [==============================] - 1078s 17ms/step - loss: 1.2011 - acc: 0.7620 - val_loss: 1.1756 - val_acc: 0.7267
Epoch 7/500
61597/61597 [==============================] - 1080s 18ms/step - loss: 1.1884 - acc: 0.7614 

61597/61597 [==============================] - 1083s 18ms/step - loss: 1.1061 - acc: 0.7587 - val_loss: 1.1339 - val_acc: 0.7672
Epoch 27/500
61597/61597 [==============================] - 1113s 18ms/step - loss: 1.1040 - acc: 0.7569 - val_loss: 1.1345 - val_acc: 0.8139
Epoch 28/500
61597/61597 [==============================] - 1111s 18ms/step - loss: 1.1032 - acc: 0.7572 - val_loss: 1.1329 - val_acc: 0.6941
Epoch 29/500
61597/61597 [==============================] - 1112s 18ms/step - loss: 1.0997 - acc: 0.7557 - val_loss: 1.1390 - val_acc: 0.7692
Epoch 30/500
61597/61597 [==============================] - 1111s 18ms/step - loss: 1.0989 - acc: 0.7564 - val_loss: 1.1332 - val_acc: 0.7661
Epoch 31/500
61597/61597 [==============================] - 1111s 18ms/step - loss: 1.0983 - acc: 0.7549 - val_loss: 1.1322 - val_acc: 0.7145
Epoch 32/500
61597/61597 [==============================] - 1113s 18ms/step - loss: 1.0970 - acc: 0.7566 - val_loss: 1.1360 - val_acc: 0.7603
Epoch 33/500
61597/

KeyboardInterrupt: 

In [135]:
#Bi-LSTM model
dth = 0.5
filter_num = 5
model = Sequential()

#embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[embedding], trainable=False))
# model.add(Dropout(dth))

#hidden layers
# model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
# model.add(Dropout(dth))
model.add(Bidirectional(LSTM(embedding_dim, dropout=0.5, recurrent_dropout=0.5)))
# model.add(Bidirectional(LSTM(embedding_dim,  batch_input_shape=(64, 39, 50), stateful=True)))
# model.add(Dropout(dth))

#output layer
model.add(Dense(modeling_data['categorical_num'], activation='softmax'))

#compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 39, 200)           4661400   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 400)               641600    
_________________________________________________________________
dense_16 (Dense)             (None, 11)                4411      
Total params: 5,307,411
Trainable params: 646,011
Non-trainable params: 4,661,400
_________________________________________________________________


In [137]:
# model.optimizer.lr=1e-4
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# early_stopping = EarlyStopping(monitor='val_acc', patience=10)

sub = "1biLSTM"
model_chk_path = 'model/cnn_hashtagcls_emb{}_weights{}.h5'.format(embedding_dim, sub)
mcp = ModelCheckpoint(model_chk_path, monitor="val_loss", verbose=1,
                      save_best_only=True, save_weights_only=False)
model.fit(X_train_u, y_train, validation_data=(X_test_u, y_test), epochs=500, batch_size=64, callbacks=[early_stopping, mcp])

Train on 61597 samples, validate on 16210 samples
Epoch 1/500
61597/61597 [==============================] - 1145s 19ms/step - loss: 1.2645 - acc: 0.7654 - val_loss: 1.2164 - val_acc: 0.7463
Epoch 2/500
61597/61597 [==============================] - 1027s 17ms/step - loss: 1.2444 - acc: 0.7668 - val_loss: 1.2055 - val_acc: 0.7378
Epoch 3/500
61597/61597 [==============================] - 871s 14ms/step - loss: 1.2319 - acc: 0.7671 - val_loss: 1.2040 - val_acc: 0.7174
Epoch 4/500
61597/61597 [==============================] - 871s 14ms/step - loss: 1.2192 - acc: 0.7658 - val_loss: 1.1901 - val_acc: 0.7734
Epoch 5/500
61597/61597 [==============================] - 870s 14ms/step - loss: 1.2092 - acc: 0.7698 - val_loss: 1.1809 - val_acc: 0.7236
Epoch 6/500
61597/61597 [==============================] - 873s 14ms/step - loss: 1.2015 - acc: 0.7681 - val_loss: 1.1750 - val_acc: 0.7559
Epoch 7/500
61597/61597 [==============================] - 871s 14ms/step - loss: 1.1945 - acc: 0.7686 - val

61597/61597 [==============================] - 867s 14ms/step - loss: 1.1349 - acc: 0.7701 - val_loss: 1.1301 - val_acc: 0.7598
Epoch 27/500
61597/61597 [==============================] - 867s 14ms/step - loss: 1.1348 - acc: 0.7727 - val_loss: 1.1281 - val_acc: 0.7533
Epoch 28/500
61597/61597 [==============================] - 865s 14ms/step - loss: 1.1322 - acc: 0.7708 - val_loss: 1.1318 - val_acc: 0.7271
Epoch 29/500
61597/61597 [==============================] - 866s 14ms/step - loss: 1.1313 - acc: 0.7697 - val_loss: 1.1289 - val_acc: 0.7350
Epoch 30/500
61597/61597 [==============================] - 861s 14ms/step - loss: 1.1297 - acc: 0.7698 - val_loss: 1.1278 - val_acc: 0.7413
Epoch 31/500
61597/61597 [==============================] - 870s 14ms/step - loss: 1.1300 - acc: 0.7705 - val_loss: 1.1274 - val_acc: 0.7017
Epoch 32/500
61597/61597 [==============================] - 896s 15ms/step - loss: 1.1270 - acc: 0.7690 - val_loss: 1.1269 - val_acc: 0.7382
Epoch 33/500
61597/61597 [

61597/61597 [==============================] - 868s 14ms/step - loss: 1.1095 - acc: 0.7694 - val_loss: 1.1166 - val_acc: 0.7342
Epoch 53/500
61597/61597 [==============================] - 878s 14ms/step - loss: 1.1102 - acc: 0.7721 - val_loss: 1.1170 - val_acc: 0.7192
Epoch 54/500
61597/61597 [==============================] - 874s 14ms/step - loss: 1.1084 - acc: 0.7701 - val_loss: 1.1155 - val_acc: 0.7292
Epoch 55/500
61597/61597 [==============================] - 875s 14ms/step - loss: 1.1081 - acc: 0.7691 - val_loss: 1.1155 - val_acc: 0.7487
Epoch 56/500
61597/61597 [==============================] - 874s 14ms/step - loss: 1.1071 - acc: 0.7714 - val_loss: 1.1180 - val_acc: 0.7352
Epoch 57/500
61597/61597 [==============================] - 873s 14ms/step - loss: 1.1068 - acc: 0.7692 - val_loss: 1.1165 - val_acc: 0.7122
Epoch 58/500
61597/61597 [==============================] - 879s 14ms/step - loss: 1.1056 - acc: 0.7686 - val_loss: 1.1147 - val_acc: 0.7371
Epoch 59/500
61597/61597 [

61597/61597 [==============================] - 890s 14ms/step - loss: 1.0961 - acc: 0.7696 - val_loss: 1.1117 - val_acc: 0.7043
Epoch 81/500
61597/61597 [==============================] - 886s 14ms/step - loss: 1.0954 - acc: 0.7710 - val_loss: 1.1098 - val_acc: 0.7192
Epoch 82/500
61597/61597 [==============================] - 884s 14ms/step - loss: 1.0946 - acc: 0.7685 - val_loss: 1.1103 - val_acc: 0.7370
Epoch 83/500
61597/61597 [==============================] - 887s 14ms/step - loss: 1.0939 - acc: 0.7688 - val_loss: 1.1091 - val_acc: 0.7333
Epoch 84/500
61597/61597 [==============================] - 886s 14ms/step - loss: 1.0941 - acc: 0.7700 - val_loss: 1.1094 - val_acc: 0.7595
Epoch 85/500
61597/61597 [==============================] - 881s 14ms/step - loss: 1.0941 - acc: 0.7684 - val_loss: 1.1110 - val_acc: 0.7276
Epoch 86/500
61597/61597 [==============================] - 883s 14ms/step - loss: 1.0929 - acc: 0.7702 - val_loss: 1.1106 - val_acc: 0.7477
Epoch 87/500
61597/61597 [

Epoch 110/500
61597/61597 [==============================] - 869s 14ms/step - loss: 1.0874 - acc: 0.7702 - val_loss: 1.1082 - val_acc: 0.7186
Epoch 111/500
61597/61597 [==============================] - 863s 14ms/step - loss: 1.0867 - acc: 0.7706 - val_loss: 1.1086 - val_acc: 0.7247
Epoch 112/500
61597/61597 [==============================] - 870s 14ms/step - loss: 1.0853 - acc: 0.7687 - val_loss: 1.1077 - val_acc: 0.7502
Epoch 113/500
61597/61597 [==============================] - 865s 14ms/step - loss: 1.0861 - acc: 0.7686 - val_loss: 1.1099 - val_acc: 0.7037
Epoch 114/500
61597/61597 [==============================] - 864s 14ms/step - loss: 1.0858 - acc: 0.7703 - val_loss: 1.1076 - val_acc: 0.7595
Epoch 115/500
61597/61597 [==============================] - 867s 14ms/step - loss: 1.0846 - acc: 0.7701 - val_loss: 1.1084 - val_acc: 0.7647
Epoch 116/500
61597/61597 [==============================] - 866s 14ms/step - loss: 1.0845 - acc: 0.7694 - val_loss: 1.1068 - val_acc: 0.7624
Epoch 

In [45]:
#class_weight
cw_map = {'cancer': 4,
 'cervicalcanc': 6,
 'gardasil': 2,
 'health': 0,
 'hpv': 9,
 'hpvvaccin': 3,
 'learntherisk': 8,
 'studi': 7,
 'vaccin': 5,
 'vaccineswork': 10,
 'vax': 1}

cw = [('hpv', 52454),
 ('vaccin', 20715),
 ('gardasil', 14558),
 ('cervicalcanc', 13457),
 ('cancer', 12978),
 ('learntherisk', 3738),
 ('health', 2939),
 ('hpvvaccin', 2724),
 ('studi', 2458),
 ('vax', 2368),
 ('vaccineswork', 2196)]

class_weight = dict()

for each in cw :
    class_weight[cw_map[each[0]]] = each[1]
class_weight

{0: 2939,
 1: 2368,
 2: 14558,
 3: 2724,
 4: 12978,
 5: 20715,
 6: 13457,
 7: 2458,
 8: 3738,
 9: 52454,
 10: 2196}

In [62]:
model.fit(X_train_u, y_train, validation_data=(X_test_u, y_test), epochs=500, batch_size=64, callbacks=[mcp, early_stopping])

Train on 61597 samples, validate on 16210 samples
Epoch 1/500
61597/61597 [==============================] - 307s 5ms/step - loss: 0.8345 - acc: 0.6914 - val_loss: 0.4316 - val_acc: 0.8578
Epoch 2/500
61597/61597 [==============================] - 270s 4ms/step - loss: 0.3395 - acc: 0.8861 - val_loss: 0.3134 - val_acc: 0.8897
Epoch 3/500
61597/61597 [==============================] - 278s 5ms/step - loss: 0.2250 - acc: 0.9251 - val_loss: 0.2898 - val_acc: 0.9068
Epoch 4/500
61597/61597 [==============================] - 305s 5ms/step - loss: 0.1572 - acc: 0.9498 - val_loss: 0.2940 - val_acc: 0.9123
Epoch 5/500
61597/61597 [==============================] - 289s 5ms/step - loss: 0.1256 - acc: 0.9599 - val_loss: 0.3136 - val_acc: 0.9141
Epoch 6/500
61597/61597 [==============================] - 269s 4ms/step - loss: 0.1026 - acc: 0.9679 - val_loss: 0.3879 - val_acc: 0.9213
Epoch 7/500
61597/61597 [==============================] - 271s 4ms/step - loss: 0.0849 - acc: 0.9736 - val_loss: 0.

#### best result
- 0.8402

#### current best model snapshot:

```python
dth = 0.5
filter_num = 5

model = Sequential()

#embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[embedding], trainable=True))

#hidden layer
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Dropout(dth))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Dropout(dth))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Dropout(dth))

#output layer
model.add(Flatten())
model.add(Dense(modeling_data['categorical_num'], activation='sigmoid'))
```

### single label model

In [51]:
#reformat the data
'''
{0: 2939,
 1: 2368,
 2: 14558,
 3: 2724,
 4: 12978,
 5: 20715,
 6: 13457,
 7: 2458,
 8: 3738,
 9: 52454,
 10: 2196}
'''
iclass_weight = {v: k for k, v in class_weight.items()}

def to_single_categorical_label(y):
    res = []
    for each in y:
        res.append(iclass_weight[max([class_weight[e] for e in each])])
    return to_categorical(res, num_classes=modeling_data['categorical_num'])

n_y = to_single_categorical_label(label)

X_train_1, X_test, y_train_1, y_test = train_test_split(data, n_y, test_size=0.2, random_state=42)

cut = int(len(X_train_1) * 0.95)
X_train = X_train_1[:cut]
y_train = y_train_1[:cut]
X_pred = X_train_1[cut:]
y_pred = y_train_1[cut:]

X_train_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_train]
X_test_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_test]
X_pred_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_pred]

X_train_u = sequence.pad_sequences(X_train_f, maxlen=seq_len)
X_test_u = sequence.pad_sequences(X_test_f, maxlen=seq_len)
X_pred_u = sequence.pad_sequences(X_pred_f, maxlen=seq_len)

print(X_train_u[:5])
print(X_test_u[:5])
print(X_pred_u[:5])
print(y_train[:5])
print(y_pred[:5])
print(y_test[:5])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     2     0    46   781    22    74
    100   121     7    13    35    55   352    54    40   162    15     1
      3     4  1126]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   364    93     5
  10430  1019    20  1770  3765    22     1     3  1637    12     9    61
     10  8726    18]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0  1159    49
    251    22    97  4382    12   448  3863     8    20     1    25   493
     18   400    13]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     2     0   122   312    90   662    57     8
    369    33    13     4   529    26   967    16  1777   831     8    39
    365    12     1]
 [    0     0     0     0   

In [54]:
dth = 0.5
filter_num = 5

model = Sequential()

#embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[embedding], trainable=True))
# model.add(Dropout(dth))

#hidden layer
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Dropout(dth))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Conv1D(embedding_dim, filter_num, padding='same', activation='relu'))
model.add(Dropout(dth))

#output layer
model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(dth))
model.add(Dense(modeling_data['categorical_num'], activation='sigmoid'))

#compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

model.fit(X_train_u, y_train, validation_data=(X_test_u, y_test), epochs=500, batch_size=64, callbacks=[mcp, early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 39, 100)           2330700   
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 39, 100)           50100     
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 39, 100)           50100     
_________________________________________________________________
dropout_15 (Dropout)         (None, 39, 100)           0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 39, 100)           50100     
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 39, 100)           50100     
_________________________________________________________________
dropout_16 (Dropout)         (None, 39, 100)           0         
__________

## train and save model for future usage

In [86]:
#save the model
model.save_weights('model/cnn_hashtagcls_emb{}_weights.h5'.format(embedding_dim))

In [ ]:
#combine train and test data and retrain the model as the final model to use for predication
early_stopping = EarlyStopping(monitor='loss', patience=5)
model.fit(
    np.concatenate((X_train_u, y_train), axis=0),
    np.concatenate((X_test_u, y_test), axis=0),
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping])
model.save_weights(
    "model/all_cnn_hashtagcls_emb{}_weights.h5".format(embedding_dim))

## extract the last hidden layer information
>https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer

In [38]:
model.layers[-1]

<function Dense.__dir__>

# Predication
**The predication will be performed on both no_labeled data set and neg_sample and pred set**

## load two data sources

In [53]:
#load data
with open("temp/neg_sample_data.json", "r", encoding="utf-8") as f:
    neg_sample = json.load(f)
    
with open("temp/no_labeled_data.json", "r", encoding="utf-8") as f:
    nolabel_sample = json.load(f)

In [54]:
print(X_pred_u[0])
print(len(X_pred_u))
print(y_pred[0])
print(len(y_pred))
pred_set = list(zip(X_pred_u, y_pred))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    3 1413
  264   41  177    1   68   32  757   37  697]
3242
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
3242


In [85]:
cw_map = {'cancer': 4,
 'cervicalcanc': 6,
 'gardasil': 2,
 'health': 0,
 'hpv': 9,
 'hpvvaccin': 3,
 'learntherisk': 8,
 'studi': 7,
 'vaccin': 5,
 'vaccineswork': 10,
 'vax': 1}



classes_decoder = {v : k for k, v in cw_map.items()}
words_decoder = dict()
for i, each in enumerate(dictionary):
    words_decoder[i] = each
# print(classes_decoder)
# print(words_decoder)

def decode_classes(data):
    true_label = []
    for i, each in enumerate(data):
        if int(each):
            true_label.append(classes_decoder[i])
    return true_label

In [ ]:
(['vaccine', 'cuts', 'teen', 'girls', "'", 'hpv', 'rates', 'by', 'two-thirds', 'health', 'living'], array([  9.89149749e-01,   8.59721843e-03,   2.24388077e-06,
         3.98910970e-06,   9.51678339e-06,   2.09738361e-03,
         2.01194634e-08,   5.34608091e-07,   1.39307536e-04,
         5.87877942e-08,   3.33531175e-10], dtype=float32), array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]))

In [74]:
print(model.predict(X_pred_u)[0])
print(model.predict_classes(X_pred_u)[0])

[  9.89149749e-01   8.59721843e-03   2.24388077e-06   3.98910970e-06
   9.51678339e-06   2.09738361e-03   2.01194634e-08   5.34608091e-07
   1.39307536e-04   5.87877942e-08   3.33531175e-10]
0


In [144]:
%%time
import csv
import operator

subs = ["2cnn+max+drop"]
for sub in subs:
    model_chk_path = 'model/cnn_hashtagcls_emb{}_weights{}.h5'.format(embedding_dim, sub)
    model = load_model(model_chk_path)
    
    pred_prob = model.predict(X_pred_u)
#     pred_class = model.predict_classes(X_pred_u)
    
    res = list(zip(X_pred, pred_prob, y_pred))
    
    output_file = "predict_res/predict_results_" + sub + ".csv"
    with open(output_file, "w", newline="", encoding="utf-8") as fw:
        header = ["tweet_text", "true_label"]
        header.extend([each[0] for each in sorted(cw_map.items(), key=operator.itemgetter(1))])
        writer = csv.DictWriter(fw, fieldnames=header)  
        writer.writeheader()
        for each in res:    
            tweet_text = " ".join(each[0])
            tweet_true_label = ", ".join(decode_classes(each[2]))
            pred_prob = each[1]
            info = [tweet_text, tweet_true_label]
            info.extend(pred_prob)
            res = {each[0]:each[1] for each in zip(header, info)}
            writer.writerow(res)

Wall time: 22 s


In [111]:
# X_train_f = [[dictionary.index(word) if word in dictionary else -1 for word in doc] for doc in X_train]
# X_train_u = sequence.pad_sequences(X_train_f, maxlen=seq_len)

X_pred_no_label_o = []

for each in nolabel_sample['data']:
    X_pred_no_label_o.append(each['words'])

X_pred_no_label = [[dictionary.index(word) if word in dictionary else 0 for word in doc] for doc in X_pred_no_label_o] 
X_pred_no_label = sequence.pad_sequences(X_pred_no_label, maxlen=seq_len)
X_pred_no_label[:5]

In [145]:
%%time
import csv
import operator

subs = ["2cnn+max+drop"]
for sub in subs:
    model_chk_path = 'model/cnn_hashtagcls_emb{}_weights{}.h5'.format(embedding_dim, sub)
    model = load_model(model_chk_path)
    
    pred_prob = model.predict(X_pred_no_label)
#     pred_class = model.predict_classes(X_pred_no_label)
    
    res = list(zip(X_pred_no_label_o, pred_prob))
    
    output_file = "predict_res/predict_results_on_no_label_sample" + sub + ".csv"
    with open(output_file, "w", newline="", encoding="utf-8") as fw:
        header = ["tweet_text", "true_label"]
        header.extend([each[0] for each in sorted(cw_map.items(), key=operator.itemgetter(1))])
        writer = csv.DictWriter(fw, fieldnames=header)  
        writer.writeheader()
        for each in res:    
            tweet_text = " ".join(each[0])
            tweet_true_label = "no label"#", ".join(decode_classes(each[2]))
            pred_prob = each[1]
            info = [tweet_text, tweet_true_label]
            info.extend(pred_prob)
            res = {each[0]:each[1] for each in zip(header, info)}
            writer.writerow(res)

Wall time: 3min 40s
